<a href="https://colab.research.google.com/github/elichen/karpathyGPT/blob/main/Let's_build_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [32]:
block_size = 8
batch_size = 32
learning_rate = 1e-3
max_inters = 5000
eval_iters = 200
eval_interval = 500
device = "cuda" if torch.cuda.is_available() else "cpu"
n_embed = 32

In [33]:
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [34]:
text = open("input.txt").read()

In [35]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size, "".join(chars)

(65, "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

In [36]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [37]:
data = torch.tensor(encode(text))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [38]:
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device), y.to(device)

In [39]:
class AttentionHead(nn.Module):
    def __init__(self, head_dim):
        super().__init__()
        self.query = nn.Linear(n_embed, head_dim)
        self.key = nn.Linear(n_embed, head_dim)
        self.value = nn.Linear(n_embed, head_dim)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        wei = q @ q.transpose(-2, -1) * k.shape[1] ** -0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        output = wei @ v
        return output

In [40]:
class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, n_embed)
        self.position_embeddings = nn.Embedding(vocab_size, n_embed)
        self.sa_head = AttentionHead(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, inputs):
        B,T = inputs.shape
        tok_emb = self.token_embeddings(inputs)
        pos_emb = self.position_embeddings(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.sa_head(x)
        logits = self.lm_head(x)
        return logits

    def calculate_loss(self, inputs, targets):
        logits = self(inputs)
        return F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

    def generate(self, input_batch, max_new_tokens):
        generated = input_batch
        for _ in range(max_new_tokens):
            logits = self(generated[:,-block_size:])
            last_logits = logits[:, -1, :]
            probabilities = F.softmax(last_logits, dim=-1)
            next_tokens = torch.multinomial(probabilities, 1).squeeze(1)
            generated = torch.cat((generated, next_tokens.unsqueeze(1)), dim=1)
        return generated

In [41]:
@torch.no_grad()
def estimate_loss(model):
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      loss = model.calculate_loss(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [42]:
model = LanguageModel().to(device)
xb,yb = get_batch("train")
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for steps in range(max_inters):
  if steps % eval_interval == 0:
    losses = estimate_loss(model)
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb,yb = get_batch("train")
  loss = model.calculate_loss(xb,yb)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
print(loss.item())

step 0: train loss 4.2023, val loss 4.2080
step 500: train loss 2.6248, val loss 2.6493
step 1000: train loss 2.5200, val loss 2.5224
step 1500: train loss 2.4869, val loss 2.4861
step 2000: train loss 2.4535, val loss 2.4640
step 2500: train loss 2.4363, val loss 2.4460
step 3000: train loss 2.4385, val loss 2.4511
step 3500: train loss 2.4331, val loss 2.4402
step 4000: train loss 2.4211, val loss 2.4332
step 4500: train loss 2.4219, val loss 2.4339
2.295100212097168


In [43]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=400)[0].tolist()))


K:
PHand ofacer ly ANGs ald I ivet qulave dr, tod wal plo:
O:
LLUR:
LANI mee.
PUME lo ans mermp fe manor s.

Wh I we nf 's.

ATAs, ontet, ng ham brury dou harep hint at VI res An che gady sthevive Pithat fours wan llo gpr sto wsonthere bbouen
th I y tear we dther:
CUKI amyofouf Glesepeet byours wif ave.
OOMUThe es?

Tuell
BREDELhond foe orud ta ty nfomears ing?

Whe cer wie glen che.
The sonass di
